In [1]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np

In [2]:
nba_teams = teams.get_teams()
df_teams = pd.DataFrame(nba_teams)
df_teams.head(60)

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,San Francisco,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


# Selección de partido
Se va a seleccionar el partido jugado el 13/12/2025 entre `okc` y `spurs`.

In [3]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import leaguegamelog

In [4]:
partidos_temporada = leaguegamefinder.LeagueGameFinder(season_nullable="2025-26")
df_partidos_temporada = partidos_temporada.get_data_frames()[0]
df_partido_okc = df_partidos_temporada[((df_partidos_temporada['GAME_DATE'] == "2025-12-13") & 
                                    (df_partidos_temporada['TEAM_ABBREVIATION'] == 'OKC'))]
df_partido_okc

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
30,22025,1610612760,OKC,Oklahoma City Thunder,0022501230,2025-12-13,SAS @ OKC,L,240,109,...,0.889,14,37,51,20,13,4,14,25,-2.0


In [9]:
game_id = df_partido_okc['GAME_ID'].iloc[0]
print(game_id)

0022501230


In [10]:
def extraer_sim(matchup):
    resultado = matchup[4]
    return resultado

def extraer_primero(matchup):
    resultado = matchup[0:3]
    return resultado

In [14]:
df_partidos_temporada['matchup_sim'] = df_partidos_temporada['MATCHUP'].loc[:].apply(extraer_sim)
df_partidos_temporada['matchup_pri'] = df_partidos_temporada['MATCHUP'].loc[:].apply(extraer_primero)
df_partidos_temporada['HA'] = np.where((df_partidos_temporada['TEAM_ABBREVIATION'] == df_partidos_temporada['matchup_pri'])&
    (df_partidos_temporada['matchup_sim'] == '@'), 'AWAY', np.where(
        (df_partidos_temporada['TEAM_ABBREVIATION'] != df_partidos_temporada['matchup_pri']) &
        (df_partidos_temporada['matchup_sim'] == 'v'),'AWAY', 'HOME'
    )
)
df_partidos_temporada.drop(columns=['matchup_sim', 'matchup_pri'], inplace=True)
df_partidos_temporada

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA
0,62025,1610612759,SAS,San Antonio Spurs,0062500001,2025-12-16,SAS @ NYK,L,240,113,...,18,24,42,29,4,5,8,13,-11.0,AWAY
1,62025,1610612752,NYK,New York Knicks,0062500001,2025-12-16,SAS @ NYK,W,240,124,...,23,36,59,27,7,6,10,20,11.0,HOME
2,22025,1610612746,LAC,LA Clippers,0022501226,2025-12-15,LAC vs. MEM,L,241,103,...,13,26,39,26,4,2,18,15,-18.0,HOME
3,22025,1610612745,HOU,Houston Rockets,0022501225,2025-12-15,HOU @ DEN,L,266,125,...,12,34,46,29,7,8,16,25,-3.0,AWAY
4,22025,1610612763,MEM,Memphis Grizzlies,0022501226,2025-12-15,MEM @ LAC,W,240,121,...,9,23,32,33,10,7,11,21,18.0,AWAY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,12025,1610612756,PHX,Phoenix Suns,0012500001,2025-10-03,PHX @ LAL,W,241,103,...,10,40,50,29,9,5,16,31,22.0,AWAY
917,12025,15016,MEL,Melbourne United,0012500009,2025-10-03,MEL @ NOP,L,240,97,...,13,35,48,23,7,1,17,18,-10.2,AWAY
918,12025,1610612747,LAL,Los Angeles Lakers,0012500001,2025-10-03,LAL vs. PHX,L,240,81,...,11,35,46,10,9,7,22,25,-22.0,HOME
919,12025,1610612752,NYK,New York Knicks,0012500008,2025-10-02,PHI @ NYK,W,238,99,...,20,38,58,19,13,2,17,26,15.0,HOME


In [15]:
partido = df_partidos_temporada[df_partidos_temporada['GAME_ID'] == game_id]
partido

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,HA
28,22025,1610612759,SAS,San Antonio Spurs,0022501230,2025-12-13,SAS @ OKC,W,241,111,...,15,37,52,21,9,6,19,23,2.0,AWAY
30,22025,1610612760,OKC,Oklahoma City Thunder,0022501230,2025-12-13,SAS @ OKC,L,240,109,...,14,37,51,20,13,4,14,25,-2.0,HOME


In [16]:
from nba_api.stats.static import players

In [18]:
players = players.get_players()
df_players = pd.DataFrame(players)
df_players

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
5098,78650,Matt Zunic,Matt,Zunic,False
5099,1641783,Tristan da Silva,Tristan,da Silva,True
5100,1628427,Vlatko Čančar,Vlatko,Čančar,False
5101,1642365,Nikola Đurišić,Nikola,Đurišić,True


In [34]:
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import boxscoretraditionalv3

In [32]:
tiros = shotchartdetail.ShotChartDetail(
    team_id= 0,
    player_id= 0,
    season_type_all_star= 'Regular Season',
    season_nullable= '2025-26'
)

df_tiros =  tiros.get_data_frames()[0]
df_tiros.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0022500001,10,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,11,26,...,Center(C),Less Than 8 ft.,1,0,13,1,1,20251021,OKC,HOU
1,Shot Chart Detail,0022500001,17,1631095,Jabari Smith Jr.,1610612745,Houston Rockets,1,11,2,...,Center(C),Less Than 8 ft.,0,-7,-5,1,1,20251021,OKC,HOU
2,Shot Chart Detail,0022500001,18,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,10,50,...,Center(C),Less Than 8 ft.,7,64,39,1,1,20251021,OKC,HOU
3,Shot Chart Detail,0022500001,27,201142,Kevin Durant,1610612745,Houston Rockets,1,10,7,...,Center(C),Less Than 8 ft.,0,0,-1,1,1,20251021,OKC,HOU
4,Shot Chart Detail,0022500001,29,1631096,Chet Holmgren,1610612760,Oklahoma City Thunder,1,9,52,...,Center(C),8-16 ft.,13,59,116,1,1,20251021,OKC,HOU


In [33]:
df_tiros.shape

(32426, 24)

In [37]:
boxscore = boxscoretraditionalv3.BoxScoreTraditionalV3(
    game_id= game_id
)
df_boxscore = boxscore.get_data_frames()[0]
df_boxscore.head()

,gameId,teamId,teamCity,teamName,teamTricode,teamSlug,personId,firstName,familyName,nameI,...,reboundsOffensive,reboundsDefensive,reboundsTotal,assists,steals,blocks,turnovers,foulsPersonal,points,plusMinusPoints
0,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1631114,Jalen,Williams,J. Williams,...,1,6,7,4,4,0,2,2,17,-18.0
1,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1631096,Chet,Holmgren,C. Holmgren,...,0,7,7,0,0,1,1,3,17,18.0
2,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1628392,Isaiah,Hartenstein,I. Hartenstein,...,8,1,9,1,1,1,3,3,10,-7.0
3,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1629652,Luguentz,Dort,L. Dort,...,0,3,3,2,0,1,0,3,3,-10.0
4,0022501230,1610612760,Oklahoma City,Thunder,OKC,thunder,1628983,Shai,Gilgeous-Alexander,S. Gilgeous-Alexander,...,0,4,4,5,0,1,5,4,29,2.0
